The implementation of prposed opionion miner is precceeded by the following setup and configuration.
-The dataset are placed in the project file.
-Insatll all the neccessary libaries and dependencies.


In [12]:
!pip install pandas matplotlib seaborn nltk spacy transformers scikit-learn torch

  Using cached torch-2.7.1-cp312-none-macosx_11_0_arm64.whl.metadata (29 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached torch-2.7.1-cp312-none-macosx_11_0_arm64.whl (68.6 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [8]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.9 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Data Ingestion

In [1]:

import os
import re
import pandas as pd


def parse_review_file(filepath, domain):
    data = []
    current_sentence = ""
    encodings = ['utf-8', 'ISO-8859-1', 'cp1252']

    for enc in encodings:
        try:
            with open(filepath, 'r', encoding=enc) as file:
                for line in file:
                    line = line.strip()
                    if not line or line.startswith("[t]") or line.startswith("***"):
                        continue

                    if line.startswith("##"):
                        current_sentence = line[2:].strip()
                    else:
                        annotations = re.findall(r"([\w\s\-&]+?)\[(\+|\-)(\d)\]", line)
                        for feature, polarity, strength in annotations:
                            data.append({
                                "domain": domain,
                                "sentence": current_sentence,
                                "feature": feature.strip(),
                                "sentiment": "positive" if polarity == "+" else "negative",
                                "strength": int(strength)
                            })
            break
        except UnicodeDecodeError:
            continue

    return pd.DataFrame(data)


def parse_all_reviews(root_folder):
    all_data = []
    output_root = "output/per_dataset"
    os.makedirs(output_root, exist_ok=True)

    for folder in os.listdir(root_folder):
        dataset_path = os.path.join(root_folder, folder)
        if not os.path.isdir(dataset_path):
            continue

        print(f"📁 Processing folder: {folder}")
        domain_data = []

        for file in os.listdir(dataset_path):
            if file.endswith(".txt"):
                path = os.path.join(dataset_path, file)
                domain = os.path.splitext(file)[0]
                df = parse_review_file(path, domain)
                domain_data.append(df)

        if domain_data:
            df_combined = pd.concat(domain_data, ignore_index=True)
            df_combined.to_csv(f"{output_root}/{folder}.csv", index=False)
            all_data.append(df_combined)

    return pd.concat(all_data, ignore_index=True)


if __name__ == "__main__":
    df_reviews = parse_all_reviews("data")
    os.makedirs("output", exist_ok=True)
    df_reviews.to_csv("output/combined_reviews.csv", index=False)
    print("✅ All reviews saved to output/combined_reviews.csv")


📁 Processing folder: Reviews-9-products
📁 Processing folder: CustomerReviews-3_domains
📁 Processing folder: Customer_review_data
✅ All reviews saved to output/combined_reviews.csv


The above code converts raw .txt to CSV by prasing.
The conversion is done and saved in separate CSVs per dataset folder.
Also adds 'domain' column for ease of anylsis and coding.

Data preprocessing

In [9]:
import pandas as pd
import re
import os
import nltk
import spacy
from nltk.corpus import stopwords


nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")

STOPWORDS = set(stopwords.words('english'))


ASPECT_SYNONYMS = {
    "battery": ["battery", "battery life", "battery backup", "batteries"],
    "screen": ["screen", "display", "lcd", "monitor"],
    "sound": ["sound", "audio", "speaker", "volume"],
    "camera": ["camera", "photo", "image", "pictures"],
    "price": ["price", "cost", "value", "worth"],
    "performance": ["performance", "speed", "slow", "fast"],
    "memory": ["memory", "storage", "ram", "space"],
    "connectivity": ["wifi", "bluetooth", "connection", "connectivity"],
    "size": ["size", "weight", "dimension"],
    "design": ["design", "look", "build", "style"]
}

REVERSE_MAP = {word: aspect for aspect, words in ASPECT_SYNONYMS.items() for word in words}

def clean_text(text, remove_stopwords=True):
    if not isinstance(text, str):
        return ""

    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()

    if remove_stopwords:
        tokens = text.split()
        text = " ".join([t for t in tokens if t not in STOPWORDS])

    return text

def normalize_feature(feature):
    return REVERSE_MAP.get(feature.lower(), feature.lower())

def extract_pos_tags(text):
    doc = nlp(text)
    return " ".join([f"{token.text}_{token.pos_}" for token in doc])

def extract_noun_phrases(text):
    doc = nlp(text)
    return [chunk.text for chunk in doc.noun_chunks]

def preprocess_dataframe(df, remove_stopwords=True):
    df["clean_sentence"] = df["sentence"].apply(lambda x: clean_text(x, remove_stopwords))
    df["clean_feature"] = df["feature"].apply(lambda x: clean_text(x, remove_stopwords=False))
    df["aspect"] = df["clean_feature"].apply(normalize_feature)


    df["pos_tags"] = df["clean_sentence"].apply(extract_pos_tags)
    df["noun_phrases"] = df["clean_sentence"].apply(extract_noun_phrases)

    return df

if __name__ == "__main__":
    df = pd.read_csv("output/combined_reviews.csv")

    print("✅ Preprocessing with stopword removal, aspect normalization, POS tagging and NP extraction...")
    df_clean = preprocess_dataframe(df, remove_stopwords=True)

    df_clean = df_clean.dropna(subset=["clean_sentence", "clean_feature", "aspect"])

    print("\n📄 Sample preview:")
    print(df_clean[["domain", "sentence", "clean_sentence", "feature", "clean_feature", "aspect", "pos_tags", "noun_phrases"]].head())

    os.makedirs("output", exist_ok=True)
    df_clean.to_csv("output/cleaned_reviews.csv", index=False)
    print("\n✅ Cleaned & enriched data saved to output/cleaned_reviews.csv")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mambaforever/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Preprocessing with stopword removal, aspect normalization, POS tagging and NP extraction...

📄 Sample preview:
   domain                                           sentence  \
0  norton                                                NaN   
1  norton  Why is it that I can install any other type of...   
2  norton  Why is it that I can install any other type of...   
3  norton  Why is it that I can install any other type of...   
4  norton  Why is it that I can install any other type of...   

                                  clean_sentence              feature  \
0                                                            software   
1  install type software installs works properly      Norton products   
2  install type software installs works properly  McAfee Anti-Virus 8   
3  install type software installs works properly               Norton   
4  install type software installs works properly               Norton   

        clean_feature              aspect  \
0            softw

In [ ]:
In this sub-task of preprocessing is done. Essential stopwords are downloaded to be used in the spaCy, aspect synonymys are set.
POS tagging and noun phrasingis also intialised in this code, these are two key aspects that helps in the next sub-suystem of product feature extraction.
Thus effective results of this code is essential nor next sub-tasks.

In [ ]:
Exploratory data analysis

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import ast

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


def run_eda(df):
    print("\U0001F4CA Dataset shape:", df.shape)
    print("\n🔍 Sample rows:")
    print(df[["sentence", "clean_sentence", "feature", "clean_feature", "sentiment", "strength"]].head())

    print("\n🎯 Sentiment distribution:")
    print(df["sentiment"].value_counts())

    print("\n💥 Sentiment strength distribution:")
    print(df["strength"].value_counts())

    print("\n🏷️ Number of unique features:", df["clean_feature"].nunique())
    print("🏷️ Number of unique aspects:", df["aspect"].nunique())
    print("🏷️ Number of unique domains:", df["domain"].nunique() if "domain" in df.columns else "N/A")

    os.makedirs("figures", exist_ok=True)

    
    sns.countplot(data=df, x="sentiment", palette="Set2")
    plt.title("Sentiment Distribution")
    plt.xlabel("Sentiment")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig("figures/sentiment_distribution.png")
    plt.clf()

   
    sns.countplot(data=df, x="strength", hue="sentiment", palette="Set1")
    plt.title("Sentiment Strength by Polarity")
    plt.xlabel("Strength (1–3)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig("figures/sentiment_strength.png")
    plt.clf()

   
    df["sentence_length"] = df["clean_sentence"].apply(lambda x: len(x.split()))
    sns.histplot(df["sentence_length"], bins=30, kde=True)
    plt.title("Sentence Length Distribution")
    plt.xlabel("Number of Words")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig("figures/sentence_length_distribution.png")
    plt.clf()

  
    if "pos_tags" in df.columns:
        print("\n\U0001F9E0 POS tag frequency (top 15):")
        pos_flat = [token.split("_")[-1] for row in df["pos_tags"].dropna() for token in row.split()]
        pos_series = pd.Series(pos_flat).value_counts().head(15)
        print(pos_series)

        sns.barplot(x=pos_series.values, y=pos_series.index, palette="magma")
        plt.title("Top 15 POS Tags")
        plt.xlabel("Frequency")
        plt.ylabel("POS Tag")
        plt.tight_layout()
        plt.savefig("figures/pos_tag_distribution.png")
        plt.clf()

    
    if "noun_phrases" in df.columns:
        print("\n🧠 Top Noun Phrases:")
        all_nps = [np for row in df["noun_phrases"].dropna() for np in row.split(";") if np.strip()]
        np_series = pd.Series(all_nps).value_counts().head(15)
        print(np_series)

        sns.barplot(x=np_series.values, y=np_series.index, palette="viridis")
        plt.title("Top 15 Noun Phrases")
        plt.xlabel("Frequency")
        plt.ylabel("Noun Phrase")
        plt.tight_layout()
        plt.savefig("figures/noun_phrase_distribution.png")
        plt.clf()

   
    if "domain" in df.columns:
        aspect_domain = df.groupby(["aspect", "domain"]).size().unstack(fill_value=0)
        print("\n📊 Aspect Mention Count by Domain:")
        print(aspect_domain.head(10))

        aspect_domain.T.plot(kind="barh", stacked=True, colormap="tab20")
        plt.title("Aspect Distribution per Domain")
        plt.xlabel("Count")
        plt.ylabel("Domain")
        plt.tight_layout()
        plt.savefig("figures/aspect_per_domain_distribution.png")
        plt.clf()

    print("\n✅ All EDA visualizations saved to /figures")


if __name__ == "__main__":
    df = pd.read_csv("output/cleaned_reviews.csv")
    df.dropna(subset=["clean_sentence", "clean_feature", "sentiment", "strength"], inplace=True)
    run_eda(df)


📊 Dataset shape: (5684, 10)

🔍 Sample rows:
                                            sentence  \
1  Why is it that I can install any other type of...   
2  Why is it that I can install any other type of...   
3  Why is it that I can install any other type of...   
4  Why is it that I can install any other type of...   
5          I bought NIS 2004 recently to try it out.   

                                  clean_sentence              feature  \
1  install type software installs works properly      Norton products   
2  install type software installs works properly  McAfee Anti-Virus 8   
3  install type software installs works properly               Norton   
4  install type software installs works properly               Norton   
5                   bought nis 2004 recently try         Installation   

        clean_feature sentiment  strength  
1     norton products  negative         3  
2  mcafee antivirus 8  positive         2  
3              norton  negative         2  
4   

/var/folders/x7/sfw2c31s1pqdjs6bbdhksdqh0000gn/T/ipykernel_55710/2132492848.py:29: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x="sentiment", palette="Set2")



🧠 POS tag frequency (top 15):
NOUN     16996
VERB      9534
ADJ       5734
ADV       3099
PROPN     2597
NUM       1970
AUX       1402
PART       689
PRON       676
ADP        525
SCONJ      268
INTJ       198
DET        158
CCONJ       59
X           39
Name: count, dtype: int64

🧠 Top Noun Phrases:
[]                                                                                   547
['phone exculsivly jabra freespeak']                                                  57
['pros']                                                                              56
['postmark date', 'second bill postmark date']                                        54
['cons']                                                                              27
['zen micro']                                                                         24
['description creatives site', 'actual battery life']                                 23
['something']                                                             

/var/folders/x7/sfw2c31s1pqdjs6bbdhksdqh0000gn/T/ipykernel_55710/2132492848.py:63: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=pos_series.values, y=pos_series.index, palette="magma")
/var/folders/x7/sfw2c31s1pqdjs6bbdhksdqh0000gn/T/ipykernel_55710/2132492848.py:78: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=np_series.values, y=np_series.index, palette="viridis")



📊 Aspect Mention Count by Domain:
domain          Apex AD2600 Progressive-scan DVD player  Canon G3  \
aspect                                                              
0                                                     0         0   
1800                                                  0         0   
2004                                                  0         0   
2004 edition                                          0         0   
2004 version                                          0         0   
3650                                                  0         0   
3d                                                    0         0   
4mp                                                   0         1   
4mp camera                                            0         1   
4mp resolution                                        0         1   

domain          Canon PowerShot SD500  Canon S100  Computer  \
aspect                                                        
0         

/var/folders/x7/sfw2c31s1pqdjs6bbdhksdqh0000gn/T/ipykernel_55710/2132492848.py:96: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()



✅ All EDA visualizations saved to /figures


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

The code visulaise and anlayis through plot few aspects of the so far cleaned data for effective product feature extraction. 
The plot are used to visulaise Sentiment distribution, Sentiment strength distribution, Distribution of sentence length, Noun Phrase Distribution and Aspect distribution across all domains.

Realtion mapping
Mapping of features into recpetive sentiments is done in two methods for the comparive study component in the assignment.
-Rule based mapping
-Model based mapping(BERT)

Rule based mapping

In [10]:

import pandas as pd
import spacy
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize
import nltk
import os

nltk.download("opinion_lexicon")
nltk.download("punkt")

nlp = spacy.load("en_core_web_sm")

POSITIVE = set(opinion_lexicon.positive())
NEGATIVE = set(opinion_lexicon.negative())

def find_opinion_word(token):
    
    for child in token.children:
        if child.pos_ == "ADJ":
            return child.text
    if token.head.pos_ == "ADJ":
        return token.head.text
    return None

def rule_based_relation_mapping(df):
    results = []

    for _, row in df.iterrows():
        sentence = row["clean_sentence"]
        aspect = row["aspect"]
        domain = row["domain"]

        if not isinstance(aspect, str) or not isinstance(sentence, str):
            continue

        doc = nlp(sentence)

        for token in doc:
            if token.text.lower() == aspect:
                opinion = find_opinion_word(token)
                if opinion:
                    sentiment = (
                        "positive" if opinion.lower() in POSITIVE else
                        "negative" if opinion.lower() in NEGATIVE else
                        "neutral"
                    )

                    results.append({
                        "domain": domain,
                        "aspect": aspect,
                        "clean_sentence": sentence,
                        "opinion_word": opinion,
                        "sentiment": sentiment,
                        "method": "rule-based"
                    })
                    break 

    return pd.DataFrame(results)


if __name__ == "__main__":
    input_path = "output/cleaned_reviews.csv"
    output_path = "output/relation_mapping_rule_based.csv"

    df = pd.read_csv(input_path)
    df.dropna(subset=["clean_sentence", "aspect", "domain"], inplace=True)

    print("🔍 Performing rule-based relation mapping...")
    result_df = rule_based_relation_mapping(df)

    os.makedirs("output", exist_ok=True)
    result_df.to_csv(output_path, index=False)
    print(f"✅ Rule-based relation mapping complete. Saved to {output_path}")


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/mambaforever/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mambaforever/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


🔍 Performing rule-based relation mapping...
✅ Rule-based relation mapping complete. Saved to output/relation_mapping_rule_based.csv


In [ ]:
The primary objective of the rule based model is to search for adjectives directly related to the feature token and mapp into it.
Certain checkpoints like code to avoid duplicate matches in the same sentence are placed for optimal results.

In [16]:
!pip show torch


Name: torch
Version: 2.7.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: filelock, fsspec, jinja2, networkx, setuptools, sympy, typing-extensions
Required-by: 


In [ ]:
Model based mapping

In [18]:

import torch
import pandas as pd
from transformers import pipeline
import os
from tqdm import tqdm

df = pd.read_csv("output/cleaned_reviews.csv")

df = df.dropna(subset=["clean_sentence", "aspect", "domain"])
df = df[df["clean_sentence"].str.len() > 5]

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

sentiment_labels = ["positive", "neutral", "negative"]

results = []
print("🔍 Predicting sentiment using BERT (BART MNLI)...")


for idx, row in tqdm(df.iterrows(), total=len(df), desc="Classifying"):
    sentence = row["clean_sentence"]
    aspect = row["aspect"]
    domain = row["domain"]

    hypothesis_template = f"The sentiment toward the {aspect} is {{}}"

    try:
        prediction = classifier(
            sequences=sentence,
            candidate_labels=sentiment_labels,
            hypothesis_template=hypothesis_template
        )
        predicted_label = prediction["labels"][0]
        confidence = prediction["scores"][0]

        results.append({
            "domain": domain,
            "aspect": aspect,
            "clean_sentence": sentence,
            "predicted_sentiment": predicted_label,
            "confidence": round(confidence, 3),
            "method": "bert-based"
        })

        if idx % 100 == 0:
            print(f"✅ Processed {idx} rows")

    except Exception as e:
        print(f"⚠️ Error on row {idx}: {e}")
        continue


os.makedirs("output", exist_ok=True)
out_path = "output/relation_mapping_bert_based.csv"
pd.DataFrame(results).to_csv(out_path, index=False)

print(f"\n✅ BERT-based relation mapping complete.")
print(f"📄 Output saved to: {out_path}")


NameError: name 'torch' is not defined

The code is contructed to load the cleaned dataset then filter out rows with usable data before load BERT-style classifier(BART).
Sentiment classes to predict are defined along adding a progress bar since the code is time complex.

Comparison of rule based mapped data and model based mapped data

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import confusion_matrix


sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (6, 4)


rule_df = pd.read_csv("output/relation_mapping_rule_based.csv")
bert_df = pd.read_csv("output/relation_mapping_bert_based.csv")


rule_df = rule_df.rename(columns={
    "sentiment": "rule_sentiment",
    "clean_sentence": "sentence"
})

bert_df = bert_df.rename(columns={
    "predicted_sentiment": "bert_sentiment",
    "clean_sentence": "sentence"
})


merged = pd.merge(rule_df, bert_df, on=["sentence", "aspect"], how="inner")


merged["agreement"] = merged["rule_sentiment"] == merged["bert_sentiment"]


total = len(merged)
agreement = merged["agreement"].sum()
agreement_rate = agreement / total if total > 0 else 0

print(f"✅ Compared {total} overlapping sentence-aspect pairs")
print(f"🤝 Agreement count: {agreement}")
print(f"📊 Agreement rate: {agreement_rate:.2%}")


os.makedirs("output", exist_ok=True)
merged.to_csv("output/comparison_result.csv", index=False)


figure_dir = "figures/comparison"
os.makedirs(figure_dir, exist_ok=True)

# Confusion Matrix
cm = confusion_matrix(merged["rule_sentiment"], merged["bert_sentiment"],
                      labels=["positive", "neutral", "negative"])

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["positive", "neutral", "negative"],
            yticklabels=["positive", "neutral", "negative"])
plt.xlabel("BERT Sentiment")
plt.ylabel("Rule-based Sentiment")
plt.title("Confusion Matrix: Rule vs BERT")
plt.tight_layout()
plt.savefig(f"{figure_dir}/confusion_matrix.png")
plt.clf()

# Agreement count plot
sns.countplot(data=merged, x="agreement", palette="coolwarm")
plt.title("Agreement Between Rule and BERT")
plt.ylabel("Count")
plt.xlabel("Agreement (True/False)")
plt.tight_layout()
plt.savefig(f"{figure_dir}/agreement_distribution.png")
plt.clf()

print("📊 Visuals saved to:")
print(f"   - {figure_dir}/confusion_matrix.png")
print(f"   - {figure_dir}/agreement_distribution.png")


The difference between the two data is analyised and comapared.
A Confusion Matrix and a Agreement count plot is generated to visulaise this comparison.

In [ ]:
Sentiment aggregation

In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from math import pi


method = "bert"  # Choose: 'bert' or 'rule'

input_paths = {
    "bert": "output/relation_mapping_bert_based.csv",
    "rule": "output/relation_mapping_rule_based.csv"
}

output_file = f"output/opinion_summary_{method}.csv"
plot_dir = f"figures/{method}_based"
os.makedirs("output", exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)


print(f"📥 Using relation mapping from: {input_paths[method]}")
df = pd.read_csv(input_paths[method])


if method == "bert":
    df = df.rename(columns={"predicted_sentiment": "sentiment"})

df = df.dropna(subset=["aspect", "sentiment"])


summary = df.groupby(["aspect", "sentiment"]).size().unstack(fill_value=0)
summary["total_mentions"] = summary.sum(axis=1)


for label in ["positive", "neutral", "negative"]:
    if label not in summary.columns:
        summary[label] = 0


for label in ["positive", "neutral", "negative"]:
    summary[f"{label}_ratio"] = summary[label] / summary["total_mentions"]


summary.to_csv(output_file)
print(f"✅ Opinion summary saved to: {output_file}")

# ---------- PLOTS ---------- #
sns.set(style="whitegrid")

# Function to annotate bars
def annotate_bars(ax):
    for p in ax.patches:
        width = p.get_width()
        ax.text(width + 0.01, p.get_y() + p.get_height()/2,
                f'{width:.2f}' if width < 1 else int(width),
                va='center')

# Top 10 Positive Aspects
plt.figure(figsize=(9, 6))
top_positive = summary.sort_values("positive_ratio", ascending=False).head(10)
ax = sns.barplot(x="positive_ratio", y=top_positive.index, data=top_positive, palette="Greens_r")
annotate_bars(ax)
plt.title(f"Top 10 Positive Aspects ({method.title()}-based)")
plt.xlabel("Positive Ratio")
plt.ylabel("Aspect")
plt.tight_layout()
plt.savefig(f"{plot_dir}/top_positive_aspects.png")
plt.clf()

# Top 10 Negative Aspects
plt.figure(figsize=(9, 6))
top_negative = summary.sort_values("negative_ratio", ascending=False).head(10)
ax = sns.barplot(x="negative_ratio", y=top_negative.index, data=top_negative, palette="Reds_r")
annotate_bars(ax)
plt.title(f"Top 10 Negative Aspects ({method.title()}-based)")
plt.xlabel("Negative Ratio")
plt.ylabel("Aspect")
plt.tight_layout()
plt.savefig(f"{plot_dir}/top_negative_aspects.png")
plt.clf()

# Top 10 Most Mentioned Aspects
plt.figure(figsize=(9, 6))
top_mentioned = summary.sort_values("total_mentions", ascending=False).head(10)
ax = sns.barplot(x="total_mentions", y=top_mentioned.index, data=top_mentioned, palette="Blues_r")
annotate_bars(ax)
plt.title(f"Top 10 Most Mentioned Aspects ({method.title()}-based)")
plt.xlabel("Total Mentions")
plt.ylabel("Aspect")
plt.tight_layout()
plt.savefig(f"{plot_dir}/top_mentioned_aspects.png")
plt.clf()

# Sentiment Composition (Stacked Bar)
plt.figure(figsize=(10, 7))
comp = top_mentioned[["positive", "neutral", "negative"]]
comp.plot(kind="barh", stacked=True, colormap="viridis")
plt.title(f"Sentiment Composition for Top Aspects ({method.title()}-based)")
plt.xlabel("Mentions")
plt.ylabel("Aspect")
plt.tight_layout()
plt.savefig(f"{plot_dir}/aspect_sentiment_composition.png")
plt.clf()

# Radar Chart for Sentiment Ratios
radar_data = summary.sort_values("total_mentions", ascending=False).head(6)
labels = ["positive_ratio", "neutral_ratio", "negative_ratio"]

angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
angles += angles[:1]  # close the loop

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

for idx, row in radar_data.iterrows():
    values = [row[label] for label in labels]
    values += values[:1]  # repeat first to close
    ax.plot(angles, values, label=idx)
    ax.fill(angles, values, alpha=0.1)

ax.set_xticks(angles[:-1])
ax.set_xticklabels([label.replace("_ratio", "").title() for label in labels])
ax.set_yticks([0.25, 0.5, 0.75])
ax.set_yticklabels(["25%", "50%", "75%"])
ax.set_title(f"Sentiment Ratios Radar Chart ({method.title()}-based)")
ax.legend(loc='upper right', bbox_to_anchor=(1.4, 1.1))
plt.tight_layout()
plt.savefig(f"{plot_dir}/aspect_sentiment_radar.png")
plt.clf()

# Overall Pie Chart
overall_counts = df["sentiment"].value_counts()
plt.figure(figsize=(6, 6))
plt.pie(overall_counts, labels=overall_counts.index, autopct="%.1f%%", colors=["green", "grey", "red"])
plt.title(f"Overall Sentiment Distribution ({method.title()}-based)")
plt.tight_layout()
plt.savefig(f"{plot_dir}/overall_sentiment_pie.png")
plt.clf()

print(f"📊 All enhanced plots saved to `{plot_dir}/`:")


A option to chose either rule maped data or the model mapped data is intialised.
The predictected is renamed to sentiment for consistency and filtered for valid rows before aggregation.
Sentiment ratios are evaluated and saved to a CSV.
Visualisation of desired plots are generated for display of results.
